# Building Up Process Scheduling Program
#### Queue1 = Round Robin List take values that has High Priority = 0 with quantum time = 3
#### Queue2 = SRJ take values that has medium Priority = 1 with quantum time = 2
#### Qeuee3 = SJF take values that has low Priority = 2 without quantum time as it's non-premptive 
# Needed Inputs
#### 1.Process Name (Pi)
#### 2.Process Arival Time 
#### 3.Prcoess Execution Time
#### 4.Queue Number of Priority


In [ ]:
def SRT():
    n = int(input("Enter the number of processes: "))

    arrival_time = []
    burst_time = []
    waiting_time = [0] * n

    print("Enter the arrival time and burst time for each process:")
    for i in range(n):
        arrival = int(input("Arrival time for process {}: ".format(i + 1)))
        burst = int(input("Burst time for process {}: ".format(i + 1)))
        arrival_time.append(arrival)
        burst_time.append(burst)

    quantum_time = 2

    remaining_time = [0] * n
    for i in range(n):
        remaining_time[i] = burst_time[i]

    t = 0  # Current time
    completed = 0  # Number of completed processes
    while completed != n:
        shortest = -1
        for i in range(n):
            if arrival_time[i] <= t and remaining_time[i] > 0:
                if shortest == -1 or remaining_time[i] < remaining_time[shortest]:
                    shortest = i

        if shortest == -1:
            t += 1
            continue

        if remaining_time[shortest] > quantum_time:
            t += quantum_time
            remaining_time[shortest] -= quantum_time
        else:
            t += remaining_time[shortest]
            waiting_time[shortest] = t - burst_time[shortest] - arrival_time[shortest]
            remaining_time[shortest] = 0
            completed += 1

    total_waiting_time = sum(waiting_time)
    average_waiting_time = total_waiting_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}".format(
                i + 1, arrival_time[i], burst_time[i], waiting_time[i]
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)


# Call the SRT function
SRT()

In [38]:
def SJF(process_data):
    total_turnaround_time = 0
    total_waiting_time = 0
    start_time = []
    exit_time = []
    s_time = 0
    process_data.sort(key=lambda x: x[1])
    """
        Sort processes according to the Arrival Time
        """
    for i in range(len(process_data)):
        ready_queue = []
        temp = []
        normal_queue = []

        for j in range(len(process_data)):
            if (process_data[j][1] <= s_time) and (process_data[j][3] == 0):
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                ready_queue.append(temp)
                temp = []
            elif process_data[j][3] == 0:
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                normal_queue.append(temp)
                temp = []

        if len(ready_queue) != 0:
            ready_queue.sort(key=lambda x: x[2])
            """
                Sort the processes according to the Burst Time
                """
            start_time.append(s_time)
            s_time = s_time + ready_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == ready_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)

        elif len(ready_queue) == 0:
            if s_time < normal_queue[0][1]:
                s_time = normal_queue[0][1]
            start_time.append(s_time)
            s_time = s_time + normal_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == normal_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)

    for i in range(len(process_data)):
        turnaround_time = process_data[i][4] - process_data[i][1]
        """
            turnaround_time = completion_time - arrival_time
            """
        total_turnaround_time = total_turnaround_time + turnaround_time
        process_data[i].append(turnaround_time)
    average_turnaround_time = total_turnaround_time / len(process_data)
    """
        average_turnaround_time = total_turnaround_time / no_of_processes
        """

    for i in range(len(process_data)):
        waiting_time = process_data[i][5] - process_data[i][2]
        """
            waiting_time = turnaround_time - burst_time
            """
        total_waiting_time = total_waiting_time + waiting_time
        process_data[i].append(waiting_time)
    average_waiting_time = total_waiting_time / len(process_data)
    """
        average_waiting_time = total_waiting_time / no_of_processes
        """
    print("Process ID\tArrival Time\tBurst Time\tTurnaround Time\tWaiting Time")
    for process in process_data:
        print(
            f"{process[0]}\t\t{process[1]}\t\t{process[2]}\t\t{process[5]}\t\t{process[6]}"
        )

    print(f"\nAverage Turnaround Time: {round(average_turnaround_time)}")
    print(f"Average Waiting Time: {round(average_waiting_time)}")


x = [[1, 0, 8, 0],[2, 1, 4, 0],[3, 2, 9 , 0],[4, 3, 5, 0]]

SJF(x)

Process ID	Arrival Time	Burst Time	Turnaround Time	Waiting Time
1		0		8		8		0
2		1		4		11		7
3		2		9		24		15
4		3		5		14		9

Average Turnaround Time: 14
Average Waiting Time: 8


In [48]:
def SRT(arrival_time, burst_time):
    n = len(arrival_time)
    waiting_time = [0] * n

    quantum_time = 2

    remaining_time = burst_time.copy()

    t = 0  # Current time
    completed = 0  # Number of completed processes
    events = []  # List to store events

    while completed != n:
        shortest = -1
        for i in range(n):
            if arrival_time[i] <= t and remaining_time[i] > 0:
                if shortest == -1 or remaining_time[i] < remaining_time[shortest]:
                    shortest = i

        if shortest == -1:
            t += 1
            continue

        if remaining_time[shortest] > quantum_time:
            t += quantum_time
            remaining_time[shortest] -= quantum_time
            events.append(f"At t = {t - quantum_time}, p{shortest} entered the kernel")
        else:
            t += remaining_time[shortest]
            waiting_time[shortest] += t - burst_time[shortest] - arrival_time[shortest]
            remaining_time[shortest] = 0
            completed += 1
            events.append(f"At t = {t}, p{shortest} left the kernel")

    total_waiting_time = sum(waiting_time)
    average_waiting_time = total_waiting_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}".format(
                i + 1, arrival_time[i], burst_time[i], waiting_time[i]
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)

    print("\nEvents:")
    for event in events:
        print(event)

    return events


def SJF(arrival_time, burst_time):
    n = len(arrival_time)
    total_turnaround_time = 0
    total_waiting_time = 0
    start_time = []
    exit_time = []
    s_time = 0
    process_data = []

    for i in range(n):
        process_data.append([i + 1, arrival_time[i], burst_time[i], 0])

    process_data.sort(key=lambda x: x[1])

    events = []  # List to store events

    for i in range(len(process_data)):
        ready_queue = []
        temp = []
        normal_queue = []

        for j in range(len(process_data)):
            if (process_data[j][1] <= s_time) and (process_data[j][3] == 0):
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                ready_queue.append(temp)
                temp = []
            elif process_data[j][3] == 0:
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                normal_queue.append(temp)
                temp = []

        if len(ready_queue) != 0:
            ready_queue.sort(key=lambda x: x[2])
            start_time.append(s_time)
            s_time = s_time + ready_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == ready_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)
            events.append(
                f"At t = {start_time[-1]}, p{process_data[k][0]} entered the kernel"
            )

        elif len(ready_queue) == 0:
            if s_time < normal_queue[0][1]:
                s_time = normal_queue[0][1]
            start_time.append(s_time)
            s_time = s_time + normal_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == normal_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)
            events.append(
                f"At t = {start_time[-1]}, p{process_data[k][0]} entered the kernel"
            )

    for i in range(len(process_data)):
        turnaround_time = process_data[i][4] - process_data[i][1]
        total_turnaround_time = total_turnaround_time + turnaround_time
        process_data[i].append(turnaround_time)

    average_turnaround_time = total_turnaround_time / len(process_data)

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time\tTurnaround Time")
    for i in range(len(process_data)):
        waiting_time = process_data[i][4] - process_data[i][1] - process_data[i][2]
        total_waiting_time = total_waiting_time + waiting_time
        print(
            "{}\t\t{}\t\t{}\t\t{}\t\t{}".format(
                process_data[i][0],
                process_data[i][1],
                process_data[i][2],
                waiting_time,
                process_data[i][4],
            )
        )

    average_waiting_time = total_waiting_time / len(process_data)

    print("\nAverage Waiting Time:", average_waiting_time)
    print("Average Turnaround Time:", average_turnaround_time)

    print("\nEvents:")
    for event in events:
        print(event)

    return events


# Input for the number of processes
n = int(input("Please enter the number of processes: "))

# Initialize lists for arrival time, burst time, and priorities
arrival_time = []
burst_time = []
priority = []

# Separate processes based on priority (0 or 1)
priority_0_processes = []
priority_1_processes = []

# Input for arrival time, burst time, and priority for each process
for i in range(n):
    arrival = int(input(f"Enter arrival time for process {i + 1}: "))
    burst = int(input(f"Enter burst time for process {i + 1}: "))
    prio = int(input(f"Enter priority (0 or 1) for process {i + 1}: "))

    arrival_time.append(arrival)
    burst_time.append(burst)
    priority.append(prio)

    # Separate processes based on priority
    if prio == 0:
        priority_0_processes.append((i + 1, arrival, burst, prio))
    elif prio == 1:
        priority_1_processes.append((i + 1, arrival, burst, prio))
    else:
        print(f"Invalid priority value for process {i + 1}. Please enter 0 or 1.")

# Display the separated processes
print("\nProcesses with Priority 0:")
print(priority_0_processes)

print("\nProcesses with Priority 1:")
print(priority_1_processes)


Processes with Priority 0:
[(1, 0, 5, 0), (4, 8, 2, 0)]

Processes with Priority 1:
[(2, 3, 6, 1), (3, 5, 4, 1)]


In [49]:
# Extracting arrival time and burst time arrays for Priority 0 processes
priority_0_arrival_time = [process[1] for process in priority_0_processes]
priority_0_burst_time = [process[2] for process in priority_0_processes]

# Extracting arrival time and burst time arrays for Priority 1 processes
priority_1_arrival_time = [process[1] for process in priority_1_processes]
priority_1_burst_time = [process[2] for process in priority_1_processes]

# SRT Scheduling for Priority 0 processes
print("\nSRT Scheduling for Priority 0:")
SRT(priority_0_arrival_time, priority_0_burst_time)

# SJF Scheduling for Priority 1 processes
print("\nSJF Scheduling for Priority 1:")
SJF(priority_1_arrival_time, priority_1_burst_time)


SRT Scheduling for Priority 0:

Process		Arrival Time	Burst Time	Waiting Time
1		0		5		0
2		8		2		0

Average Waiting Time: 0.0

Events:
At t = 0, p0 entered the kernel
At t = 2, p0 entered the kernel
At t = 5, p0 left the kernel
At t = 10, p1 left the kernel

SJF Scheduling for Priority 1:

Process		Arrival Time	Burst Time	Waiting Time	Turnaround Time
1		3		6		0		9
2		5		4		4		13

Average Waiting Time: 2.0
Average Turnaround Time: 7.0

Events:
At t = 3, p1 entered the kernel
At t = 9, p2 entered the kernel


['At t = 3, p1 entered the kernel', 'At t = 9, p2 entered the kernel']

In [ ]:
def SRT(arrival_time, burst_time, priority):
    n = len(arrival_time)
    waiting_time = [0] * n
    remaining_time = burst_time.copy()
    t = 0  # Current time
    completed = 0  # Number of completed processes
    events = []  # List to store events

    while completed != n:
        shortest = -1
        for i in range(n):
            if arrival_time[i] <= t and remaining_time[i] > 0:
                if shortest == -1 or remaining_time[i] < remaining_time[shortest]:
                    shortest = i

        if shortest == -1:
            t += 1
            continue

        t += 1
        remaining_time[shortest] -= 1

        if remaining_time[shortest] == 0:
            waiting_time[shortest] = t - burst_time[shortest] - arrival_time[shortest]
            completed += 1
            events.append(
                f"At t = {t}, p{shortest} leaved the kernel and entered the queue"
            )
        else:
            events.append(f"At t = {t}, p{shortest} is in the kernel")

    total_waiting_time = sum(waiting_time)
    average_waiting_time = total_waiting_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}".format(
                i + 1, arrival_time[i], burst_time[i], waiting_time[i]
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)

    print("\nEvents:")
    for event in events:
        print(event)

    return events


def SJF(arrival_time, burst_time):
    n = len(arrival_time)
    total_turnaround_time = 0
    total_waiting_time = 0
    start_time = []
    exit_time = []
    s_time = 0
    process_data = []

    for i in range(n):
        process_data.append([i + 1, arrival_time[i], burst_time[i], 0])

    process_data.sort(key=lambda x: x[1])

    events = []  # List to store events

    for i in range(len(process_data)):
        ready_queue = []
        temp = []
        normal_queue = []

        for j in range(len(process_data)):
            if (process_data[j][1] <= s_time) and (process_data[j][3] == 0):
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                ready_queue.append(temp)
                temp = []
            elif process_data[j][3] == 0:
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                normal_queue.append(temp)
                temp = []

        if len(ready_queue) != 0:
            ready_queue.sort(key=lambda x: x[2])
            start_time.append(s_time)
            s_time = s_time + ready_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == ready_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)
            events.append(
                f"At t = {start_time[-1]}, p{process_data[k][0]} entered the kernel"
            )

        elif len(ready_queue) == 0:
            if s_time < normal_queue[0][1]:
                s_time = normal_queue[0][1]
            start_time.append(s_time)
            s_time = s_time + normal_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == normal_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)
            events.append(
                f"At t = {start_time[-1]}, p{process_data[k][0]} entered the kernel"
            )

    for i in range(len(process_data)):
        turnaround_time = process_data[i][4] - process_data[i][1]
        total_turnaround_time = total_turnaround_time + turnaround_time
        waiting_time = turnaround_time - process_data[i][2]
        total_waiting_time = total_waiting_time + waiting_time

    average_waiting_time = total_waiting_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}".format(
                process_data[i][0],
                process_data[i][1],
                process_data[i][2],
                process_data[i][4] - process_data[i][1] - process_data[i][2],
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)

    print("\nEvents:")
    for event in events:
        print(event)

    return events


# Take inputs for processes
n = int(input("Enter the number of processes: "))
processes = []

for i in range(n):
    arrival = int(input(f"Enter arrival time for process P{i + 1}: "))
    burst = int(input(f"Enter burst time for process P{i + 1}: "))
    priority = int(input(f"Enter priority (0 or 1) for process P{i + 1}: "))

    process = {"arrival_time": arrival, "burst_time": burst, "priority": priority}
    processes.append(process)

# Separate processes based on priority
sjf_processes = []
other_processes = []

for process in processes:
    if process["priority"] == 1:
        sjf_processes.append(process)
    else:
        other_processes.append(process)

# Call the SJF function with processes having priority 1
if sjf_processes:
    SJF(sjf_processes)

# Call the other algorithm function with processes having priority 0
if other_processes:
    SRT(other_processes)